In [1]:
import tensorflow as tf
import numpy as np
import nltk
from nltk.corpus import wordnet as wn
from gensim.models import KeyedVectors
from gensim.parsing.preprocessing import remove_stopwords
import collections
from collections import defaultdict
import os
import re
import random
import itertools
import requests
import pickle
import heapq
import copy
from functools import reduce
import math
import pdb
from gpt2.src.score import score_model
from gpt2.src.generate_prompt import generate_prompt
from gpt2.src.encoder import get_encoder
import pickle

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
from gpt2.src.generate_poem_naive import generate_poem_naive
res = generate_poem_naive(model_name="117M", length=100, nsamples=10, batch_size=10, raw_text="There was a clever boy whose name is James.", should_rhyme=True)


# Generate poems naively by trying for multiple times


INFO:tensorflow:Restoring parameters from gpt2/models/117M/model.ckpt
======================================== SAMPLE 1 VALID 0 ========================================
======================================== SAMPLE 2 VALID 0 ========================================
======================================== SAMPLE 3 VALID 0 ========================================
======================================== SAMPLE 4 VALID 0 ========================================
======================================== SAMPLE 5 VALID 0 ========================================
======================================== SAMPLE 6 VALID 0 ========================================
======================================== SAMPLE 7 VALID 0 ========================================
======================================== SAMPLE 8 VALID 0 ========================================
======================================== SAMPLE 9 VALID 0 ========================================
=======================================

======================================== SAMPLE 83 VALID 0 ========================================
======================================== SAMPLE 84 VALID 0 ========================================
======================================== SAMPLE 85 VALID 0 ========================================
======================================== SAMPLE 86 VALID 0 ========================================
======================================== SAMPLE 87 VALID 0 ========================================
======================================== SAMPLE 88 VALID 0 ========================================
======================================== SAMPLE 89 VALID 0 ========================================
======================================== SAMPLE 90 VALID 0 ========================================
======================================== SAMPLE 91 VALID 0 ========================================
======================================== SAMPLE 92 VALID 0 ========================================


======================================== SAMPLE 165 VALID 0 ========================================
======================================== SAMPLE 166 VALID 0 ========================================
======================================== SAMPLE 167 VALID 0 ========================================
======================================== SAMPLE 168 VALID 0 ========================================
======================================== SAMPLE 169 VALID 0 ========================================
======================================== SAMPLE 170 VALID 0 ========================================
======================================== SAMPLE 171 VALID 0 ========================================
======================================== SAMPLE 172 VALID 0 ========================================
======================================== SAMPLE 173 VALID 0 ========================================
======================================== SAMPLE 174 VALID 0 ===============================

In [5]:
def save_poem_to_text_file(poems, file_name):
    f = open(file_name,'a') 
    for poem in poems:
        for line in poem:
            line = re.sub(r'[^\w\s]','',line)
            f.write(line)
            f.write("\n")
        f.write("\n")
save_poem_to_text_file(res, 'py_files/saved_objects/silly_poems_3.txt')

In [23]:
# poem = pickle.load('gpt2.pkl')
f = open('gpt2.pkl', 'rb')
poems = pickle.load(f)

In [24]:
print(poems)

[262, 37934, 994, 29742, 866, 438, 198, 3152, 15133, 4692, 290, 288, 260, 560, 26, 198, 1870, 1793, 9859, 1474, 284, 607, 438, 64, 2415, 11, 198, 1890, 1793, 290, 607, 1672, 11386, 13, 8117, 1752, 373, 281, 21848, 2415, 3706, 37392]


In [2]:

import re
f = open('py_files/saved_objects/silly_poems_with_templates.txt', 'rb')
for line in f:
    line = str(line).split(',')
    for i in range(len(line)):
        line[i] = re.sub(r"[^A-Za-z]+", '', line[i])
    line[0] = line[0][1:]
    line[-1] = line[-1][:-2]
    print(' '.join(line))

there once was a woman enjoying her life in lamar
who had men in a car
i took up one tune
of a new moon
and was in a th bar

there was an enthusiastic man that lived in dakar
who saw things from another star
i found out one afternoon
that the dark moon
and lost in the dark scar

there once was an enthusiastic woman named romaine
who told stories about the pain
she felt off one kind
of a high wind
and fell off a high plane

there once was an enthusiastic man named case
who did wonders for the birthplace
i got a report
from a local court
and asked for a special grace

there once was an enthusiastic woman named germaine
who laid eggs on a grain
i came up one mind
in a un kind
and lay in a u vein

there once was an enthusiastic woman named germaine
who wrote books in every domain
she experienced off one mankind
in any great whirlwind
and fell in a great rain

there was an enthusiastic man that lived in moraine
who collected funds for this campaign
he operated out two whirlwind
for any good

# Save poems in a csv for Amazon Turk

In [17]:
from pandas import DataFrame

In [28]:
# Specify path
in_path = 'py_files/saved_objects/limericks.txt'
out_path = 'py_files/saved_objects/csv/good_poems.csv'

In [30]:
in_txt = open(in_path, 'rb')
lines = {'line1': [],
         'line2': [],
         'line3': [],
         'line4': [],
         'line5': []}

counter = 0
for line in in_txt:
    line = line.rstrip().decode('utf-8')
    if len(line) < 5:
        continue
    lines['line' + str(counter+1)].append(line)
    counter += 1
    counter %= 5
df = DataFrame(lines, columns= ['line1', 'line2', 'line3', 'line4', 'line5'])
export_csv = df.to_csv (out_path, index = None, header=True)


In [9]:
from py_files.Limericks import Limerick_Generate
lg = Limerick_Generate(model_name='117M')

In [81]:
lg.gen_first_line_new('sarah', contains_adjective=True, strict=True, search_space=10, seed='magic')

[['there', 'was', 'a', 'clever', 'witch', 'named', 'sarah'],
 ['there', 'was', 'a', 'warrior', 'named', 'sarah'],
 ['there', 'once', 'was', 'a', 'girl', 'named', 'sarah'],
 ['there', 'was', 'a', 'lucky', 'witch', 'named', 'sarah'],
 ['there', 'was', 'a', 'silly', 'witch', 'named', 'sarah'],
 ['there', 'was', 'a', 'cunning', 'witch', 'named', 'sarah'],
 ['there', 'was', 'a', 'magician', 'named', 'sarah']]

In [16]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [73]:
print(nlp("queen").similarity(nlp("sarah")))

0.3652998840991959


In [5]:
import requests

In [8]:
api_url = 'https://api.datamuse.com/words'
w_response = {dic["word"] for dic in requests.get(api_url, params={'rel_syn': "grace"}).json()}
print(w_response)

{'favour', 'decorate', 'seemliness', 'blessing', 'beautify', 'embellish', 'good will', 'state of grace', 'honor', 'goodwill', 'deck', 'adorn', 'favor', 'thanksgiving', 'honour', 'ornament', 'gracility'}
